In [ ]:
# !pip install deepface
# !pip install opencv-python
# !pip install scikit-image
# !pip install onnxruntime insightface


In [17]:
import cv2
import numpy as np
from skimage import io


In [18]:
alg = "haarcascade_frontalface_default.xml"
haar_cascade = cv2.CascadeClassifier(alg)
file_name = "data/955px-NASA_Astronaut_Group_18.jpg"

img = cv2.imread(file_name, 0)
grey_img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

faces = haar_cascade.detectMultiScale(grey_img, scaleFactor=1.15, minNeighbors=3, minSize=(60, 60))


In [10]:
faces.shape

(17, 4)

In [8]:
i = 0

for x,y,w,h in faces:
    cropped_image = img[y:y+h, x: x+w]
    target_file_name = 'stored_faces/' + str(i) + '.jpg'
    cv2.imwrite(target_file_name, cropped_image)
    i = i+1
    


Some reference  
YuNet
https://medium.com/pythons-gurus/what-is-the-best-face-detector-ab650d8c1225  
https://learnopencv.com/what-is-face-detection-the-ultimate-guide/  


In [18]:
!wget https://github.com/astaileyyoung/CineFace/raw/main/research/data/face_detection_yunet_2023mar.onnx -P ./model/



--2024-08-07 05:13:32--  https://github.com/astaileyyoung/CineFace/raw/main/research/data/face_detection_yunet_2023mar.onnx
Resolving github.com (github.com)... 20.205.243.166
Connecting to github.com (github.com)|20.205.243.166|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/astaileyyoung/CineFace/main/research/data/face_detection_yunet_2023mar.onnx [following]
--2024-08-07 05:13:33--  https://raw.githubusercontent.com/astaileyyoung/CineFace/main/research/data/face_detection_yunet_2023mar.onnx
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 232589 (227K) [application/octet-stream]
Saving to: ‘./model/face_detection_yunet_2023mar.onnx.1’

face_detection_yune 100%[===================>] 227.14K  

In [19]:
class FaceDetectorYunet():
    def __init__(self, model_path='./face_detection_yunet_2023mar.onnx',
                 img_size=(400, 400),
                 threshold=0.5):
        self.model_path = model_path
        self.img_size = img_size
        self.fd = cv2.FaceDetectorYN_create(str(model_path),
                                            "",
                                             img_size,
                                             score_threshold=threshold)


    def draw_faces(self,
                   image,
                   faces,
                   draw_landmarks=False,
                   show_confidence=False):
        for face in faces:
            color = (0, 0, 255)
            thickness = 2
            cv2.rectangle(image, (face['x1'], face['y1']), (face['x2'], face['y2']), color, thickness, cv2.LINE_AA)

            if draw_landmarks:
                landmarks = face['landmarks']
                for landmark in landmarks:
                    radius = 5
                    thickness = -1
                    cv2.circle(image, landmark, radius, color, thickness, cv2.LINE_AA)

            if show_confidence:
                confidence = face['confidence']
                confidence = "{:.2f}".format(confidence)
                position = (face['x1'], face['y1'] - 10)
                font = cv2.FONT_HERSHEY_SIMPLEX
                scale = 0.5
                thickness = 2
                cv2.putText(image, confidence, position, font, scale, color, thickness, cv2.LINE_AA)
        return image

    def scale_coords(self, image, prediction):
        ih, iw = image.shape[:2]
        rw, rh = self.img_size
        a = np.array([
                (prediction['x1'], prediction['y1']),
                (prediction['x1'] + prediction['x2'], prediction['y1'] + prediction['y2'])
                    ])
        b = np.array([iw/rw, ih/rh])
        c = a * b
        prediction['img_width'] = iw
        prediction['img_height'] = ih
        prediction['x1'] = int(c[0,0].round())
        prediction['x2'] = int(c[1,0].round())
        prediction['y1'] = int(c[0,1].round())
        prediction['y2'] = int(c[1,1].round())
        prediction['face_width'] = (c[1,0] - c[0,0])
        prediction['face_height'] = (c[1,1] - c[0,1])
        # prediction['face_width'] = prediction['x2'] - prediction['x1']
        # prediction['face_height'] = prediction['y2'] - prediction['y1']
        prediction['area'] = prediction['face_width'] * prediction['face_height']
        prediction['pct_of_frame'] = prediction['area']/(prediction['img_width'] * prediction['img_height'])
        return prediction

    def detect(self, image):
        if isinstance(image, str):
            image = cv2.imread(str(image))
        gray_img = cv2.cvtColor(image, cv2.COLOR_BGRA2BGR)
        img = cv2.resize(gray_img, self.img_size)
        self.fd.setInputSize(self.img_size)
        _, faces = self.fd.detect(img)
        if faces is None:
            return None
        else:
            predictions = self.parse_predictions(image, faces)
            return predictions

    def parse_predictions(self, image, faces):
        data = []
        for num, face in enumerate(list(faces)):
            x1, y1, x2, y2 = list(map(int, face[:4]))
            landmarks = list(map(int, face[4:len(face)-1]))
            landmarks = np.array_split(landmarks, len(landmarks) / 2)
            positions = ['left_eye', 'right_eye', 'nose', 'right_mouth', 'left_mouth']
            landmarks = {positions[num]: x.tolist() for num, x in enumerate(landmarks)}
            confidence = face[-1]
            datum = {'x1': x1,
            'y1': y1,
            'x2': x2,
            'y2': y2,
            'face_num': num,
            'landmarks': landmarks,
            'confidence': confidence,
            'model': 'yunet'}
        d = self.scale_coords(image, datum)
        data.append(d)
        return data


In [20]:
class FaceDetectorYunet():
    def __init__(self,
                  model_path='./face_detection_yunet_2023mar.onnx',
                  img_size=(300, 300),
                  threshold=0.5):
        self.model_path = model_path
        self.img_size = img_size
        self.fd = cv2.FaceDetectorYN_create(str(model_path),
                                            "",
                                            img_size,
                                            score_threshold=threshold)

    def draw_faces(self,
                   image,
                   faces,
                   draw_landmarks=False,
                   show_confidence=False):
        for face in faces:
            color = (0, 0, 255)
            thickness = 2
            cv2.rectangle(image, (face['x1'], face['y1']), (face['x2'], face['y2']), color, thickness, cv2.LINE_AA)

            if draw_landmarks:
                landmarks = face['landmarks']
                for landmark in landmarks:
                    radius = 5
                    thickness = -1
                    cv2.circle(image, landmark, radius, color, thickness, cv2.LINE_AA)

            if show_confidence:
                confidence = face['confidence']
                confidence = "{:.2f}".format(confidence)
                position = (face['x1'], face['y1'] - 10)
                font = cv2.FONT_HERSHEY_SIMPLEX
                scale = 0.5
                thickness = 2
                cv2.putText(image, confidence, position, font, scale, color, thickness, cv2.LINE_AA)
        return image

    def scale_coords(self, image, prediction):
        ih, iw = image.shape[:2]
        rw, rh = self.img_size
        a = np.array([
                (prediction['x1'], prediction['y1']),
                (prediction['x1'] + prediction['x2'], prediction['y1'] + prediction['y2'])
                    ])
        b = np.array([iw/rw, ih/rh])
        c = a * b
        prediction['img_width'] = iw
        prediction['img_height'] = ih
        prediction['x1'] = int(c[0,0].round())
        prediction['x2'] = int(c[1,0].round())
        prediction['y1'] = int(c[0,1].round())
        prediction['y2'] = int(c[1,1].round())
        prediction['face_width'] = (c[1,0] - c[0,0])
        prediction['face_height'] = (c[1,1] - c[0,1])
        # prediction['face_width'] = prediction['x2'] - prediction['x1']
        # prediction['face_height'] = prediction['y2'] - prediction['y1']
        prediction['area'] = prediction['face_width'] * prediction['face_height']
        prediction['pct_of_frame'] = prediction['area']/(prediction['img_width'] * prediction['img_height'])
        return prediction

    def detect(self, image):
        if isinstance(image, str):
            image = cv2.imread(str(image))
        img = cv2.cvtColor(image, cv2.COLOR_BGRA2BGR)
        img = cv2.resize(img, self.img_size)
        self.fd.setInputSize(self.img_size)
        _, faces = self.fd.detect(img)
        if faces is None:
            return None
        else:
            predictions = self.parse_predictions(image, faces)
            return predictions

    def parse_predictions(self,
                          image,
                          faces):
        data = []
        for num, face in enumerate(list(faces)):
            x1, y1, x2, y2 = list(map(int, face[:4]))
            landmarks = list(map(int, face[4:len(face)-1]))
            landmarks = np.array_split(landmarks, len(landmarks) / 2)
            positions = ['left_eye', 'right_eye', 'nose', 'right_mouth', 'left_mouth']
            landmarks = {positions[num]: x.tolist() for num, x in enumerate(landmarks)}
            confidence = face[-1]
            datum = {'x1': x1,
                     'y1': y1,
                     'x2': x2,
                     'y2': y2,
                     'face_num': num,
                     'landmarks': landmarks,
                     'confidence': confidence,
                     'model': 'yunet'}
            d = self.scale_coords(image, datum)
            data.append(d)
        return data


In [7]:
import IPython
def show_image(image):
    _, ret = cv2.imencode('.jpg', image)
    i = IPython.display.Image(data=ret)
    IPython.display.display(i)

In [22]:
fd = FaceDetectorYunet()

In [ ]:
img = io.imread('https://github.com/astaileyyoung/CineFace/blob/main/research/notebooks/images/img_1.jpg?raw=true')
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
faces = fd.detect(img)
if faces:
    fd.draw_faces(img, faces)
show_image(img)

In [ ]:
# img = io.imread('https://github.com/astaileyyoung/CineFace/blob/main/research/notebooks/images/img_2.jpg?raw=true')
img = io.imread('./data/img_2.jpg')
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
faces = fd.detect(img)
if faces:
    fd.draw_faces(img, faces)
show_image(img)

In [ ]:
file_name = "data/636px-NASA_Astronaut_Group_18.jpg"
file_name = "data/img_2.jpg"

# img = cv2.imread(file_name, 0)
img = io.imread(file_name)
# img = io.imread('https://github.com/astaileyyoung/CineFace/blob/main/research/notebooks/images/img_2.jpg?raw=true')
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

fd = FaceDetectorYunet()
faces = fd.detect(img)
if faces:
    fd.draw_faces(img, faces)
show_image(img)

In [48]:
len(faces)

1

In [ ]:
import matplotlib.pyplot as plt



plt.figure(figsize=(20,10))
plt.imshow(img)
plt.axis('off')

Reference : https://medium.com/@yongsun.yoon/nba-face-recognition-system-345034ffed8c

In [1]:
import insightface
from insightface.app.common import Face
from insightface.model_zoo import model_zoo
import cv2


det_model_path = 'buffalo_s/det_500m.onnx'
rec_model_path = 'buffalo_s/w600k_mbf.onnx'

det_model = model_zoo.get_model(f'./model/{det_model_path}')
# rec_model = model_zoo.get_model(f'./model/{rec_model_path}')

det_model.prepare(ctx_id=0, input_size=(640, 640), det_thres=0.5)

Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}


/home/aiap/miniconda3/envs/cv/lib/python3.11/site-packages/onnxruntime/capi/onnxruntime_inference_collection.py:69: UserWarning: Specified provider 'CUDAExecutionProvider' is not in available provider names.Available providers: 'AzureExecutionProvider, CPUExecutionProvider'
  warnings.warn(


In [2]:
file_name = ["data/636px-NASA_Astronaut_Group_18.jpg", "data/955px-NASA_Astronaut_Group_18.jpg","data/1432px-NASA_Astronaut_Group_18.jpg", "data/foreign-workers-5.jpg"]

img = cv2.imread(file_name[1])

bboxes, kpss = det_model.detect(img, max_num=0, metric='default')

In [3]:
def draw_faces(image,
               faces,
               draw_landmarks=False,
               show_confidence=False):
    for face in faces:
        color = (0, 0, 255)
        thickness = 2
        cv2.rectangle(image, (int(face[0]), int(face[1])), (int(face[2]), int(face[3])), color, thickness, cv2.LINE_AA)

        if draw_landmarks:
            landmarks = face['landmarks']
            for landmark in landmarks:
                radius = 5
                thickness = -1
                cv2.circle(image, landmark, radius, color, thickness, cv2.LINE_AA)

        if show_confidence:
            confidence = face[4]
            confidence = "{:.2f}".format(confidence)
            position = (int(face[0]), int(face[1] - 10))
            font = cv2.FONT_HERSHEY_SIMPLEX
            scale = 0.5
            thickness = 2
            cv2.putText(image, confidence, position, font, scale, color, thickness, cv2.LINE_AA)
    return image

In [ ]:
# cv2.putText(img=img,'Test',(100,60))

In [4]:
img = draw_faces(img,bboxes, show_confidence=True)

In [9]:

len(bboxes)
# len(kpss)
# kpss[0]
bbox = bboxes[0, :4]

len(bbox)
det_score = bboxes[0, 4]

det_score

0.91072

In [13]:
kpss[0]

array([[201.9659 , 116.19924],
       [231.1289 , 118.66086],
       [217.08386, 133.66927],
       [202.01231, 147.81348],
       [227.38632, 149.73907]], dtype=float32)

In [27]:

# face = bboxes[0]
color = (0, 0, 255)
# thickness = 2
# cv2.rectangle(img, (int(face[0]), int(face[1])), (int(face[2]), int(face[3])), color, thickness, cv2.LINE_AA)

cv2.circle(img=img, center=(201,116), radius=3, color=color, thickness=2)
cv2.circle(img=img, center=(231,118), radius=2, color=(0, 255, 0), thickness=2)
cv2.circle(img=img, center=(217, 134), radius=2, color=(0, 0, 255), thickness=2)

array([[[ 37,  49,  61],
        [ 40,  52,  64],
        [ 41,  53,  65],
        ...,
        [ 21,  30,  33],
        [ 21,  30,  33],
        [ 21,  30,  33]],

       [[ 40,  52,  64],
        [ 42,  54,  66],
        [ 40,  52,  64],
        ...,
        [ 25,  34,  37],
        [ 26,  35,  38],
        [ 26,  35,  38]],

       [[ 42,  54,  66],
        [ 42,  54,  66],
        [ 40,  52,  64],
        ...,
        [ 28,  37,  40],
        [ 28,  37,  40],
        [ 27,  36,  39]],

       ...,

       [[ 93, 101, 100],
        [ 83,  91,  90],
        [ 83,  91,  90],
        ...,
        [ 75,  87,  89],
        [ 79,  88,  91],
        [ 72,  81,  84]],

       [[ 86,  94,  93],
        [ 83,  91,  90],
        [ 80,  88,  87],
        ...,
        [ 79,  89,  89],
        [ 82,  90,  90],
        [ 79,  87,  87]],

       [[ 77,  85,  84],
        [ 86,  94,  93],
        [ 80,  88,  87],
        ...,
        [ 82,  92,  92],
        [ 87,  95,  95],
        [ 90,  98,  98]]

In [5]:
import IPython
def show_image(image):
    _, ret = cv2.imencode('.jpg', image)
    i = IPython.display.Image(data=ret)
    IPython.display.display(i)

In [24]:
bboxes[0]

array([181.54738 ,  88.133606, 245.36903 , 173.82637 ,   0.91072 ],
      dtype=float32)

In [ ]:
show_image(img)